In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

In [5]:
customer_df = pd.read_csv('Wholesale_customers_data.csv')
customer_df.drop(['Channel', 'Region'], axis=1, inplace=True)

In [6]:
customer_df.shape
print(customer_df)

     Fresh   Milk  Grocery  Frozen  Detergents_Paper  Delicatessen
0    12669   9656     7561     214              2674          1338
1     7057   9810     9568    1762              3293          1776
2     6353   8808     7684    2405              3516          7844
3    13265   1196     4221    6404               507          1788
4    22615   5410     7198    3915              1777          5185
5     9413   8259     5126     666              1795          1451
6    12126   3199     6975     480              3140           545
7     7579   4956     9426    1669              3321          2566
8     5963   3648     6192     425              1716           750
9     6006  11093    18881    1159              7425          2098
10    3366   5403    12974    4400              5977          1744
11   13146   1124     4523    1420               549           497
12   31714  12319    11757     287              3881          2931
13   21217   6208    14982    3095              6707          

In the next few notebooks, we are going to do some Unsupervised Exploration of the `customer` table in our Database.

> What does a data scientist do? PCA on the `customer` table. - Joshua Cook

# Basic Stats

In [7]:
from scipy.stats import skew

In [8]:
skew(customer_df)

array([  2.55258269,   4.03992212,   3.57518722,   5.88782573,
         3.61945758,  11.11353365])

In [9]:
import random 

random.sample(range(10), 2)

[0, 9]

In [10]:
stats = customer_df.describe().T
stats['skew'] = skew(customer_df)
stats

,count,mean,std,min,25%,50%,75%,max,skew
Fresh,440.0,12000.297727,12647.328865,3.0,3127.75,8504.0,16933.75,112151.0,2.552583
Milk,440.0,5796.265909,7380.377175,55.0,1533.00,3627.0,7190.25,73498.0,4.039922
Grocery,440.0,7951.277273,9503.162829,3.0,2153.00,4755.5,10655.75,92780.0,3.575187
Frozen,440.0,3071.931818,4854.673333,25.0,742.25,1526.0,3554.25,60869.0,5.887826
Detergents_Paper,440.0,2881.493182,4767.854448,3.0,256.75,816.5,3922.00,40827.0,3.619458
Delicatessen,440.0,1524.870455,2820.105937,3.0,408.25,965.5,1820.25,47943.0,11.113534


# Sampling the Dataset 

In this notebook, we begin to explore the `customer` table by sampling the table. First, let's sample three random points and examine them. 

In [11]:
np.random.seed(42)

In [12]:
sample = customer_df.sample(3)

In [13]:
sample

,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicatessen
265,5909,23527,13699,10155,830,3636
78,10766,1175,2067,2096,301,167
347,27380,7184,12311,2809,4621,1022


In [14]:
stats

,count,mean,std,min,25%,50%,75%,max,skew
Fresh,440.0,12000.297727,12647.328865,3.0,3127.75,8504.0,16933.75,112151.0,2.552583
Milk,440.0,5796.265909,7380.377175,55.0,1533.00,3627.0,7190.25,73498.0,4.039922
Grocery,440.0,7951.277273,9503.162829,3.0,2153.00,4755.5,10655.75,92780.0,3.575187
Frozen,440.0,3071.931818,4854.673333,25.0,742.25,1526.0,3554.25,60869.0,5.887826
Detergents_Paper,440.0,2881.493182,4767.854448,3.0,256.75,816.5,3922.00,40827.0,3.619458
Delicatessen,440.0,1524.870455,2820.105937,3.0,408.25,965.5,1820.25,47943.0,11.113534


# Sampling for a Statistical Description

We are able to take the mean and standard deviation of the data, but what if we want to visualize it? 

Of course, this dataset is small, but we might want techniques that work even when the dataset is very large.

Let's start by looking at 1% of the data. 

In [15]:
sample_1pct_1 = customer_df.sample(5)

In [16]:
sample_1pct_1.mean()

Fresh               14123.6
Milk                 5801.0
Grocery              5475.8
Frozen               2812.6
Detergents_Paper     1097.8
Delicatessen         1987.6
dtype: float64

### How does this compare to the actual mean?

In [17]:
sample_1pct_1.mean() - stats['mean']

Fresh               2123.302273
Milk                   4.734091
Grocery            -2475.477273
Frozen              -259.331818
Detergents_Paper   -1783.693182
Delicatessen         462.729545
dtype: float64

Let's think about this in terms of the standard deviations.

In [18]:
(sample_1pct_1.mean() - stats['mean'])/stats['std']

Fresh               0.167885
Milk                0.000641
Grocery            -0.260490
Frozen             -0.053419
Detergents_Paper   -0.374108
Delicatessen        0.164082
dtype: float64

### How does it do?


### Let's try it again

In [19]:
sample_1pct_2 = customer_df.sample(100)

In [20]:
sample_1pct_2.mean() - stats['mean']

Fresh              -630.227727
Milk                 43.014091
Grocery            -617.077273
Frozen             -147.991818
Detergents_Paper   -240.953182
Delicatessen       -324.960455
dtype: float64

In [21]:
(sample_1pct_2.mean() - stats['mean'])/stats['std']

Fresh              -0.049831
Milk                0.005828
Grocery            -0.064934
Frozen             -0.030484
Detergents_Paper   -0.050537
Delicatessen       -0.115230
dtype: float64

### How does it do?

### Repeatedly Sample

Let's do it 10 times.

In [22]:
sample_means = []
for _ in range(10):
    sample_means.append(customer_df.sample(5).mean())

sample_means = np.array(sample_means)
(sample_means.mean(axis=0)-stats['mean'])/stats['std']

Fresh              -0.100475
Milk                0.085239
Grocery            -0.010988
Frozen              0.000051
Detergents_Paper    0.062445
Delicatessen       -0.155884
dtype: float64

And 50 times.

In [23]:
sample_means = []
for _ in range(50):
    sample_means.append(customer_df.sample(5).mean())

sample_means = np.array(sample_means)
(sample_means.mean(axis=0)-stats['mean'])/stats['std']

Fresh              -0.001005
Milk                0.014526
Grocery            -0.009778
Frozen              0.080827
Detergents_Paper   -0.029035
Delicatessen        0.037616
dtype: float64

And 100 times.

In [24]:
sample_means = []
for _ in range(100):
    sample_means.append(customer_df.sample(5).mean())

sample_means = np.array(sample_means)
(sample_means.mean(axis=0)-stats['mean'])/stats['std']

Fresh              -0.056725
Milk               -0.032065
Grocery            -0.039822
Frozen              0.003006
Detergents_Paper   -0.040477
Delicatessen        0.030977
dtype: float64

### What do we notice?

### Take a larger sample

Totally different. Which makes sense ... we're only taking 1% of the data!

What if we take a sample of 10% of the data?

In [25]:
sample_10pct_1 = customer_df.sample(44)
(sample_10pct_1.mean() - stats['mean'])/stats['std']

Fresh              -0.050512
Milk                0.049586
Grocery             0.128665
Frozen             -0.105639
Detergents_Paper    0.087743
Delicatessen       -0.009383
dtype: float64

### Is this sample good enough for plotting?

https://stats.stackexchange.com/questions/2541/is-there-a-reference-that-suggest-using-30-as-a-large-enough-sample-size

In [ ]:
sns.pairplot(sample_10pct_1, kind='reg')